In [1]:
import os
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
# Seleccionar el archivo necesario
# df = pd.read_hdf('chb02.hdf')
df = pd.read_hdf('data/devices/DSI-4/chb02.hdf')

In [3]:
df

,FP1-F7,FP2-F8,frame,window,test,seizure
0,-2.100122e-04,9.865690e-05,0.0,0.0,0,0
1,1.953602e-07,1.953602e-07,1.0,0.0,0,0
2,1.953602e-07,1.953602e-07,2.0,0.0,0,0
3,1.953602e-07,1.953602e-07,3.0,0.0,0,0
4,1.953602e-07,1.953602e-07,4.0,0.0,0,0
...,...,...,...,...,...,...
7675,-2.949939e-05,2.285714e-05,921595.0,238.0,2,0
7676,-1.777778e-05,1.621490e-05,921596.0,238.0,2,0
7677,-5.645910e-05,1.074481e-05,921597.0,238.0,2,0
7678,-3.262515e-05,1.699634e-05,921598.0,238.0,2,0


In [4]:
maximoRegistros = int(df['test'].max()) #Para obtener el número mayor y saber cuantos registros hay en los archivos

In [5]:
registros = []
for i in range(maximoRegistros):
    registros.append(df.loc[df['test'] == i]) #Acomodar los valores en una lista 

In [6]:
registros[0]

,FP1-F7,FP2-F8,frame,window,test,seizure
0,-2.100122e-04,9.865690e-05,0.0,0.0,0,0
1,1.953602e-07,1.953602e-07,1.0,0.0,0,0
2,1.953602e-07,1.953602e-07,2.0,0.0,0,0
3,1.953602e-07,1.953602e-07,3.0,0.0,0,0
4,1.953602e-07,1.953602e-07,4.0,0.0,0,0
...,...,...,...,...,...,...
7675,-6.446886e-06,-4.004884e-05,241915.0,61.0,0,0
7676,1.758242e-06,-4.239316e-05,241916.0,61.0,0,0
7677,-9.768010e-07,-4.590965e-05,241917.0,61.0,0,0
7678,4.884005e-06,-4.317460e-05,241918.0,61.0,0,0


In [7]:
def es_ataque(ventana):
    framesSeizure = ventana.loc[ventana['seizure']==1]
    #print(f'Ventana {ventana}')   
    tamañoVentana = len(ventana) #Número de frames de la ventana actual
    #print(f'Frames en la ventana {tamañoVentana}')        
    tamañoframes=len(framesSeizure) #Número de frames con seizure=1
    #print(f'Frames en la ventana con seizure=1 {tamañoframes}')
        
    porcentajeVentana = tamañoVentana*0.5
    return tamañoframes >= porcentajeVentana

In [8]:
dfAtaques = pd.DataFrame()

for i in range(maximoRegistros):
    maxVentana = int(registros[i]['window'].max())
    #print(f'--------------------Registro {i+1}--------------------')
    registro = registros[i]
    for ventana in range(maxVentana):
        ventana_actual = registro.loc[registro['window'] == ventana]
        ataque = es_ataque(ventana_actual)
        
#         Agregar más procesamiento aquí
        
        renglon = {
            
#             Incluir los resultados aquí

            'window': ventana,
            'test': i,
            'seizure': ataque,
        }
        dfAtaques = dfAtaques.append(renglon, ignore_index=True)

In [9]:
# Optimizar memoria
types = {
    'window': 'Int16',
    'test': 'Int16',
    'seizure': 'bool',
}

dfAtaques = dfAtaques.astype(types)

In [10]:
dfAtaques.loc[dfAtaques['seizure'] == 1]

,seizure,test,window
8,True,0,8
9,True,0,9
10,True,0,10
11,True,0,11
12,True,0,12
13,True,0,13
259,True,1,198
260,True,1,199
261,True,1,200
262,True,1,201
